<a href="https://colab.research.google.com/github/KellyPared/deep-learning-challenge/blob/main/randomForestclassifieripynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Forest and Keras Tuner

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report




In [5]:
#Import and read the charity_data.csv.
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
#df.head()

In [6]:
# drop any rows with missing values
df.dropna(inplace=True)

In [7]:
# encode any object columns
le = LabelEncoder()
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = le.fit_transform(df[col])


In [8]:
X = df.drop('IS_SUCCESSFUL', axis=1)  # select all columns except the target variable
y = df['IS_SUCCESSFUL']  # select the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # split the data


In [9]:
rfc = RandomForestClassifier(n_estimators=100, random_state=42)  # create the classifier with 100 trees


In [10]:
rfc.fit(X_train, y_train)  # fit the model to the training data


RandomForestClassifier(random_state=42)

In [11]:
# evaluate the model on the testing set
accuracy = rfc.score(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

Test Accuracy: 0.77


# Calculate precision, recall, and F1-score for your random forest classifier:

In [12]:

# make predictions on the test set
y_pred = rfc.predict(X_test)

# print classification report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.79      0.69      0.73      3196
           1       0.75      0.84      0.79      3664

    accuracy                           0.77      6860
   macro avg       0.77      0.76      0.76      6860
weighted avg       0.77      0.77      0.76      6860



Keras Tuner

In [3]:
 !pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.5 MB/s eta 0:00:00


In [13]:
from kerastuner.tuners import RandomSearch
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import HyperParameters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

<ipython-input-13-84993c643b34>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [14]:


def build_model(hp):
    '''This makes the model for the tuner and passes in hp'''
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_dim=input_dim))
    model.add(layers.Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(layers.Dense(units=output_dim, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [15]:
hp = HyperParameters()


In [16]:
input_dim = 11
output_dim = 10
model = Sequential()
model.add(Dense(12, input_dim=input_dim, activation='relu'))
model.add(Dense(output_dim, activation='sigmoid'))


In [17]:

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='my_project')


In [19]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [20]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))



Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
416               |416               |units
0.1               |0                 |dropout
0.0018603         |0.0072823         |learning_rate

Epoch 1/5


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/hypermodel.py", line 144, in fit
    return model.fit(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    rais

RuntimeError: ignored

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))
